### dotenv 관련

In [1]:
import os
import requests
import random
import platform
import gradio as gr
from io import BytesIO
from PIL import Image, ImageDraw, ImageFont
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("AZURE_VISION_API_KEY")
object_detection_endpoint = os.getenv("OBJECT_DETECTION_ENDPOINT")
vision_endpoint = os.getenv("VISION_ENDPOINT")

c:\Users\EL91\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### jupyter 환경 에서 gradio 돌릴때 eventloop 중첩 안되게 하는 설정

In [2]:
## jupyter 에서 gradio 돌릴때 eventloop 중첩 안되게 하는 설정
import nest_asyncio
nest_asyncio.apply()

In [ ]:
def random_color():
  return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) 


def get_font():
  font_size = 20
  try:
      if platform.system() == "Windows":
          # 윈도우: 맑은 고딕
          return ImageFont.truetype("malgun.ttf", font_size)
      elif platform.system() == "Darwin":  # macOS
          # 맥: 애플 고딕
          return ImageFont.truetype("AppleGothic.ttf", font_size)
      else:  # Linux 등
          # 기본 폰트 (한글 지원 안 될 수 있음)
          return ImageFont.load_default(size=font_size)
  except IOError:
      # 지정한 폰트 파일이 없을 경우 PIL 기본 폰트 사용
      return ImageFont.load_default()

def request_image_analysis(image_url, selected_features, options=dict()) :
    endpoint = vision_endpoint
    # method : POST

    params = {
        "features" : ",".join(selected_features),
        "api-version" : "2024-02-01",
        # "smartcrops-aspect-ratios" : "0.75, 1.0, 1.33, 1.5, 1.77",
        # "gender-neutral-caption" : True
    }

    params.update(options)

    headers = {
        "Ocp-Apim-Subscription-Key" : api_key
    }

    body = {
        "url" : image_url
    }

    response = requests.post(endpoint, params=params, headers=headers, json=body)

    if response.status_code != 200 :
        print("Error:", response.status_code, response.text)
        return None

    return response.json()

def draw_image(image_url, data, threshold) :

    response = requests.get(image_url)

    image = Image.open(BytesIO(response.content))
    draw = ImageDraw.Draw(image)

    drawable_features = ["objects", "denseCaptions", "smartCrops"]

    for feature in drawable_features :

        feature_key = '{}Result'.format(feature)

        if feature_key not in data.keys() :
            continue

        block_list = data['{}Result'.format(feature)]['values']

        for block in block_list :

            color=random_color()

            bounding_box = block['boundingBox']
            x, y, w, h = bounding_box['x'], bounding_box['y'], bounding_box['w'], bounding_box['h']

            if feature == "objects" :
                name = block['tags'][0]['name']
                confidence = block['tags'][0]['confidence']
            elif feature == "denseCaptions" :
                name = block['text']
                confidence = block['confidence']
            else :
                name = ""
                confidence = 1.0

            if confidence < threshold:
                continue

            draw.rectangle([(x, y), (x + w, y + h)], outline=color, width=3)
            draw.text((x, y+h), f"{name} ({confidence * 100:.2f}%)", fill=color, font=get_font())

    return image

# send("https://cdn.pixabay.com/photo/2017/10/17/21/15/pair-2862155_1280.jpg")

with gr.Blocks() as demo :

    FEATURES = ["objects", "caption", "denseCaptions", "read", "tags", "smartCrops"]

    def click_send(image_url, threshold, selected_features, is_neutral, ratio_text) :

        # print("Selected features : ", ",".join(selected_features))

        options = dict()

        if "caption" in selected_features or "denseCaptions" in selected_features :
            options.update({
                "gender-neutral-caption" : is_neutral
            })
        
        if "smartCrops" in selected_features and ratio_text.strip() != "" :
            options.update({
                "smartcrops-aspect-ratios" : ratio_text
            })

        print("Options : ", options)

        response_data = request_image_analysis(image_url, selected_features, options)
        result_image = draw_image(image_url, response_data, threshold)

        return result_image, response_data
    
    def change_features(selected_features, is_neutral, ratio_text) :
        
        print("Selected features : ", selected_features, is_neutral, ratio_text)
        selected_caption = False
        selected_smart_crops = False

        if "caption" in selected_features or "denseCaptions" in selected_features :
            selected_caption = True
        
        if "smartCrops" in selected_features :
            selected_smart_crops = True

        return gr.update(visible=selected_caption), gr.update(visible=selected_smart_crops)

    gr.Markdown("### Image Analysis")

    features_checkbox = gr.CheckboxGroup(label="기능 선택", choices=FEATURES, value=["objects"])

    gender_radio = gr.Radio(label="성별 중립", choices=[("중립", True), ("구분", False)], value=False, visible=False)
    ratio_text = gr.Textbox(label="smart crops 비율", placeholder="ex) 0.75, 1.2, 1.8", visible=False)

    image_url_text = gr.Textbox(label="Image URL")
    thres_slider = gr.Slider(minimum=0, maximum=1, value=0.5, step=0.01, label="Threshold",interactive=True)
    send_button = gr.Button("전송")

    with gr.Row() :
        output_image = gr.Image(label="결과 이미지", type="pil")
        output_text = gr.JSON(label="결과 데이터")

    send_button.click(click_send, inputs=[image_url_text, thres_slider, features_checkbox, gender_radio, ratio_text], outputs=[output_image, output_text])
    features_checkbox.change(change_features, inputs=[features_checkbox, gender_radio, ratio_text], outputs=[gender_radio, ratio_text])

demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Selected features :  ['objects', 'caption'] False None
Selected features :  ['objects', 'caption', 'denseCaptions'] False None
Selected features :  ['objects', 'caption', 'denseCaptions', 'smartCrops'] False None
Selected features :  ['caption', 'denseCaptions', 'smartCrops'] False 
Options :  {'gender-neutral-caption': False, 'smartcrops-aspect-ratios': '0.75, 1.1, 1.8'}
Selected features :  ['caption', 'smartCrops'] False 0.75, 1.1, 1.8
Options :  {'gender-neutral-caption': False, 'smartcrops-aspect-ratios': '0.75, 1.1, 1.8'}
Options :  {'gender-neutral-caption': True, 'smartcrops-aspect-ratios': '1.5'}
Options :  {'gender-neutral-caption': True, 'smartcrops-aspect-ratios': '1.0'}
Selected features :  ['caption', 'smartCrops', 'denseCaptions'] True 1.0
Options :  {'gender-neutral-caption': True, 'smartcrops-aspect-ratios': '1.0'}
Selected features :  ['caption', 'smartCrops', 'denseCaptions', 'objects'] True 1.0
Selected features :  ['smartCrops', 'denseCaptions', 'objects'] True 1.0